# Illustrating of co-plotting `hemibrain` skeleton with flywire dataset

## Step 1: Import neccesary library modules now

In [1]:
import navis
import fafbseg
import flybrains

In [2]:
from pyroglancer.localserver import startdataserver, closedataserver
from pyroglancer.flywire import flywireurl2dict, add_flywirelayer

In [3]:
import navis.interfaces.neuprint as neu
from navis.interfaces.neuprint import NeuronCriteria as NC, SynapseCriteria as SC

## Step 2: Set configurations to fetch from data from `neuprint`

In [4]:
client = neu.Client('https://neuprint.janelia.org/', dataset='hemibrain:v1.1')

In [5]:
client

Client("https://neuprint.janelia.org", "hemibrain:v1.1")

## Step 3: Get sample skids and neuropil meshes from `neuprint`

In [6]:
neurons_df, roi_counts_df = neu.fetch_neurons(NC(status='Traced',type='DA1_lPN',regex=True))

In [7]:
da1pnbodyidList = neurons_df[['bodyId']]

In [8]:
da1pn_skel = neu.fetch_skeletons(da1pnbodyidList, heal=1000, max_threads=30)

In [9]:
#fetch mesh for Right Antenna lobe..
al_R = neu.fetch_roi('AL(R)')

In [10]:
hemibrain_mesh = navis.Volume(flybrains.JRCFIB2018Fraw.mesh)

## Step 4: Transform them to Flywire space..

In [11]:
flywire_neuron=navis.xform_brain(da1pn_skel,source='JRCFIB2018Fraw', target='FLYWIRE')

INFO  : Pre-caching deformation field(s) for transforms... (navis)


Transform path: JRCFIB2018Fraw -> JRCFIB2018F -> JRCFIB2018Fum -> JRC2018F -> FAFB14um -> FAFB14 -> FAFB14raw -> FLYWIREraw -> FLYWIRE


In [12]:
flywirevol = {}
flywirevol['hemibrain']=navis.xform_brain(hemibrain_mesh, source='JRCFIB2018Fraw', target='FLYWIRE')
flywirevol['AL_R']=navis.xform_brain(al_R, source='JRCFIB2018Fraw', target='FLYWIRE')

Transform path: JRCFIB2018Fraw -> JRCFIB2018F -> JRCFIB2018Fum -> JRC2018F -> FAFB14um -> FAFB14 -> FAFB14raw -> FLYWIREraw -> FLYWIRE
Transform path: JRCFIB2018Fraw -> JRCFIB2018F -> JRCFIB2018Fum -> JRC2018F -> FAFB14um -> FAFB14 -> FAFB14raw -> FLYWIREraw -> FLYWIRE


In [13]:
flywirevol['hemibrain'].id = 200
flywirevol['AL_R'].id = 300
flywirevol

{'hemibrain': <navis.Volume(name=None, id=200, color=(0.85, 0.85, 0.85, 0.2), vertices.shape=(22125, 3), faces.shape=(44270, 3))>,
 'AL_R': <navis.Volume(name=None, id=300, color=(0.85, 0.85, 0.85, 0.2), vertices.shape=(38829, 3), faces.shape=(77672, 3))>}

## Step 5: Start the dataserver to host precomputed data..

In [14]:
startdataserver()

Serving data from:  /var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpy3bd6f06
Serving directory at http://127.0.0.1:8000


## Step 6: Get the url from flywire to append data onto..

In [15]:
shorturl = 'https://ngl.flywire.ai/?json_url=https://globalv1.flywire-daf.com/nglstate/5644175227748352'
#This flywire segments has some stuff similar to the DA1 PNs, you will notice their closeness in the final plot

## Step 7: Add bodyids to flywire layers..

In [16]:
tmpviewer = add_flywirelayer(flywireurl2dict(shorturl), layer_kws = {'type': 'skeletons',
                                                                     'source': flywire_neuron[2:4],
                                                                     'color': 'red'}) 
#'alpha': 0.3, doesn't work yet in skeleton layers..

/private/var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpy3bd6f06/precomputed/skeletons/754534424
/private/var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpy3bd6f06/precomputed/skeletons/5813039315
creating: /private/var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpy3bd6f06/precomputed/skeletons/seg_props
{754534424: ['#ff0000'], 5813039315: ['#ff0000']}
flywire url at: https://ngl.flywire.ai/?json_url=https://globalv1.flywire-daf.com/nglstate/4891270581321728


## Step 8: Add neuropil meshes to flywire layers

In [17]:
shorturl = tmpviewer

In [18]:
tmpviewer = add_flywirelayer(flywireurl2dict(shorturl), layer_kws = {'type': 'volumes',
                                                                     'source': [flywirevol['hemibrain'],
                                                                                flywirevol['AL_R']],
                                                                     'color': ['grey', 'magenta'], 
                                                                     'alpha': 0.3})

mesh/200
Seg id is: 200
Full filepath: /private/var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpy3bd6f06/precomputed/volumes/mesh/200
mesh/300
Seg id is: 300
Full filepath: /private/var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpy3bd6f06/precomputed/volumes/mesh/300
creating: /private/var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpy3bd6f06/precomputed/volumes/mesh/segment_properties
creating: /private/var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpy3bd6f06/precomputed/volumes/mesh/segment_names
flywire url at: https://ngl.flywire.ai/?json_url=https://globalv1.flywire-daf.com/nglstate/5696265044099072


127.0.0.1 - - [04/Feb/2021 21:04:48] "GET /precomputed/skeletons/info HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2021 21:04:48] "GET /precomputed/volumes/mesh/info HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2021 21:04:48] "GET /precomputed/volumes/mesh/200:0 HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2021 21:04:48] "GET /precomputed/volumes/mesh/300:0 HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2021 21:04:48] "GET /precomputed/skeletons/754534424 HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2021 21:04:48] "GET /precomputed/skeletons/5813039315 HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2021 21:04:48] "GET /precomputed/volumes/mesh/200 HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2021 21:04:48] "GET /precomputed/volumes/mesh/300 HTTP/1.1" 200 -


# Screenshot of the flywire instance should like below..

### The grey mesh is the hemibrain neuropil, magenta is the Antenna lobe (Right), you can see the skeletons (red) plotted in the skeleton layer, and some similar neurons (meshes) of the flywire dataset plotted in green and yellowas well

![alt text](pics/hemibrain2flywire.png "Title")

## Step 9: Close the dataserver

In [ ]:
closedataserver()